In [1]:
### Bibliotecas básicas de gestión de datos
import pandas as pd
import numpy as np

### Bibliotecas gráficas
import matplotlib.pyplot as plt
import seaborn as sns 

### Utilidades Scikit-learn 
from sklearn.model_selection import train_test_split

### Métricas
from sklearn import metrics
from sklearn.metrics import accuracy_score, roc_curve, auc

### Modelos
from xgboost import XGBClassifier, plot_importance

########################################################
### Para un flujo de trabajo más rápido, las bibliotecas de Interpretabilidad
### se instalarán/cargarán sobre la marcha del tutorial
########################################################

### Add-ons adicionales
import warnings
warnings.filterwarnings('ignore')
plt.style.use('fivethirtyeight')
%matplotlib inline

In [3]:
from sklearn import preprocessing
import pandas as pd
import numpy as np

#leemos archivo de pokemon
df = pd.read_csv("Datos/pokemon2.csv", nrows = 100000)
df_fights = pd.read_csv("Datos/combats.csv", nrows = 100000)

#Transformación de las columnas Type 1 y Type 2 con datos tipo String en columnas de tipos de pokemon con booleanos
types1 = pd.get_dummies(df["Type 1"])
types2 = pd.get_dummies(df["Type 2"])
pd.set_option('max_rows', None)
pd.reset_option('max_rows')
types = types1 | types2

df_clean = pd.merge(df, types, left_index=True, right_index=True)

df_clean.dropna(thresh=28,inplace=True)
df_clean.drop(["Type 1", "Type 2"], axis = 1,inplace= True)

# Se cambia el nombre de las columnas del dataframe de combates
df_fights.rename(columns = {'First_pokemon':'ID1', 'Second_pokemon':'ID2',
 'Winner': 'isFirstWinner'}, inplace = True)

#Se cambia el valor del la columna ganadora por un true o un false dependiendo de si ha ganado el pokemon de la primera columna o no
df_fights.loc[df_fights['isFirstWinner'] == df_fights['ID1'], 'isFirstWinner'] = True
df_fights.loc[df_fights['isFirstWinner'] == df_fights['ID2'], 'isFirstWinner'] = False
df_fights.dropna(inplace=True)

# Igualamos el nombre de la columna del id a ID1
df_clean.rename(columns = {'#':'ID1'}, inplace= True)
df_clean.columns
df_fights.columns

#Añadimos todos los campos del 1er pokemon a la tabla
df_fights = df_fights.merge(df_clean, on='ID1')
df_fights

# Igualamos el nombre de la columna del id a ID2
df_clean.rename(columns = {'ID1':'ID2'}, inplace= True)
#Añadimos todos los campos del 2º pokemon a la tabla
df_fights = df_fights.merge(df_clean, on='ID2')

#El resultado de los ultimos dos merges es un dataframe cuyas columnas son
#los datos de dos pokemon en una misma linea y si el primero de ellos es el ganador del combate o no

df_fights = df_fights.drop(["ID1","ID2"], axis = 1)

#cambiamos true y false por ints 1 y 0 por comodidad
df_fights["isFirstWinner"] = df_fights["isFirstWinner"].astype(int)

df_scaled =df_fights.drop(["Name_x","Name_y"], axis=1)


df_scaled = df_scaled.sample(frac = 1)
dftest = df_scaled.tail(30000)
dfrest = df_scaled.head(20000)

In [4]:
# Crear el objetivo y la columna de características y dividir el conjunto de datos en conjunto de prueba y de entrenamiento.

X = df_scaled.iloc[:, :-1]
y = df_scaled.iloc[:, -1]

# 75% para entrenamiento, 25% para test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=0)
X_train.shape, X_test.shape

((37500, 52), (12500, 52))